# Challenge problem: neutrino oscillations

This notebook is meant to get you started towards implementing the neutrino oscillation procedure detailed in http://arxiv.org/abs/1904.10559. The paper shows the details for both 2- and 3-flavour oscillations, but we'll focus here on just two, $\nu_e$ and $\nu_\mu$. Specifically, we will work on reproducing Figure 2, where we will plot the 'survival' probability of the electron neutrino.

### Derivations and important physical constants

Recall from the lectures that I showed an expression for the probability of $\nu_e$ turning into a $\nu_\mu$:
\begin{equation}
P_{\nu_e \rightarrow \nu_\mu} = |\nu_\mu(t)|^2 = \left[ \sin(2\theta) \sin \left( \frac{E_2 - E_1}{2\hbar}t \right) \right]^2
\end{equation}

Oftentimes this is expressed in terms of travel distance $L$ and a neutrino energy $E$, using the correspondence $E_i = |p|^2 c^2 + m_i^2 c^4$ and making some approximations, such as $L \approx ct$ for particles travelling at close to the speed of light:
\begin{eqnarray}
P_{\nu_e \rightarrow \nu_\mu} =  |\nu_\mu(t)|^2 &=& \left[ \sin(2\theta) \sin \left( \frac{(m_{\nu_\mu}^2  - m^2_{\nu_e})}{2\hbar} \frac{L}{E} \right) \right]^2 \\
&=&  \left[ \sin(2\theta) \sin \left( \frac{\Delta m^2}{2\hbar c} \frac{L}{E} \right) \right]^2 
\end{eqnarray}
where $\Delta m^2$ is shorthand for $m_{\nu_\mu}^2  - m^2_{\nu_e}$.
For a full derivation of this, see, for examples,  Griffith's book _Introduction to Elementary Particles_. I'm just going to take it as a given.

**Task 1.** Track down the relevant constants you need: $\theta$, $\Delta m^2$ for electron and muon neutrinos, and $L$. You'll have to be really careful with units here, making sure you don't pick up (or lose) any rogue factors of $c$, etc. The physical neutrino parameters can be found online; the value of $L$ they used was from the KamLAND experiment (https://arxiv.org/pdf/hep-ex/0212021.pdf).


In [ ]:
θ = 0.5867 # rad, Found on Wikipedia discussion of neutrino oscillation and PMNS matrix

# Thanks to the authors Carlos and Benjamin for helping me out with this part!
Δm_sq = 7.37e-5  # expressed in (eV / c^2)^2
hbar_c = 1.97e-10 # expressed in eV * km
L = 180 # km, from the experimental paper

gev_conversion_factor = (1e-9)**2  / 1e-9 

# This gives us a prefactor with units such that we can input an energy in GeV
prefactor = (Δm_sq * L / (2 * hbar_c)) * gev_conversion_factor

**Task 2.** Set up the required registers.

For two-flavour oscillation, we only need to use a single qubit. So, we just need one quantum register with one qubit, and a classical register with a classical output bit.

In [ ]:
import numpy as np

from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit
from qiskit import execute, BasicAer

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
q = QuantumRegister(1)
c = ClassicalRegister(1)

**Task 3.** Write a function that will apply a circuit to your registers that (a) creates the initial superposition, (b) applies the evolution operator for a neutrino of specified energy, (c) measures the output.

The way this is done in the paper is actually by using $|0\rangle$ and $|1\rangle$ to represent the flavour basis (instead of using them to represent the mass basis). This way our measurements are made in the flavour basis as well. So make sure you are applying the unitary basis transformations in the "right" direction.

In [ ]:
def oscillate(E, q, c):
    circuit = QuantumCircuit(q, c)
    
    # First we convert from flavour basis to mass basis
    circuit.u3(2*θ, 0, 0, q[0])
    
    # Next we apply the evolution operator
    circuit.u1(prefactor / E, q[0])

    # Then we undo the basis change
    circuit.u3(-2*θ, 0, 0, q[0])
    
    circuit.measure(q, c)
    
    return circuit

**Task 4.** Simulate your circuit for varying energies, and plot the results.

In [ ]:
energy_range = list(np.linspace(2, 12, 200))
probs = []
n_shots = 10000

for E in energy_range:
    neutrino_circuit = oscillate(E / 1000, q, c)
    
    backend = BasicAer.get_backend('qasm_simulator') # the device to run on
    result = execute(neutrino_circuit, backend, shots=n_shots).result()
    counts = result.get_counts(neutrino_circuit)
    
    # Extract the probabilities
    if '0' in counts.keys():
        p_0 = counts['0'] / n_shots
        p_1 = 1 - p_0
    else:
        p_1 = counts['1'] / n_shots
        p_0 = 1 - p_1
        
    probs.append(p_0)    